In [1]:
#Tech Task: Task 1

#Steps: Write the code to perform an ETL process to extract a data set from the supplied source and persist in an 
# accessible format. Load the data in a database of your choice and write the queries to get the below data 
# outputs.  Visualise the results in the tooling of your choice

#Output: List the Top 5 Peanut Butters based in Australia and sorted them by highest Energy content per 100g List 
#the Top 10 Countries together with the number of products with Zinc content above ‘0.001’ and that have more than one product
#Grouping product categories by those that contain Chicken, Pork and Tofu list their Average, Median and Standard 
# Deviation Protein content per 100g, excluding data that is not available (NaN)


import pandas as pd
import numpy as np
import sqlite3
import csv
import urllib
from zipfile import ZipFile

In [9]:
#Retrieve, save and unzip file
url = "https://s3-ap-southeast-2.amazonaws.com/vibrato-data-test-public-datasets/world-food-facts.zip"
urllib.request.urlretrieve (url, "world-food-facts.zip")
zf = ZipFile("world-food-facts.zip")
zf.extractall(path = './')
zf.close()

In [3]:
foodTSV = 'en.openfoodfacts.org.products.tsv'

In [13]:
#open database file and csv reader (formatted for reading tsv)

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database

reader = csv.reader(open(foodTSV), delimiter='\t')
headers = reader.__next__()
print(len(headers))

162


In [15]:
# Create table in sqlite3.  First 64 entries have type text, the rest have type real.  

do_this_sqlite_1 = "CREATE TABLE IF NOT EXISTS FoodFacts ( "

for headerName in headers[:64]:
    do_this_sqlite_1 += " " + headerName + " text,"

for headerName in headers[64:]:
    do_this_sqlite_1 += " " + headerName + " real,"
    
do_this_sqlite_1 = do_this_sqlite_1[:-1]

do_this_sqlite_1 += ");"

do_this_sqlite_1 = do_this_sqlite_1.replace('-', '_') #sql doesn't seem to like hyphens

# print(do_this_sqlite_1)

myCursor.execute(do_this_sqlite_1)

In [14]:
#Insert values from tsv file into table

do_this_sqlite = "INSERT INTO FoodFacts VALUES ( "
do_this_sqlite += "?," * len(headers)
do_this_sqlite = do_this_sqlite[:-1]
do_this_sqlite += ")"

print(do_this_sqlite)

rowErrors = 0

for row in reader:
    try:
        myCursor.execute(do_this_sqlite,row)
    except:
        rowErrors += 1
        
print("rowErrors = {}".format(rowErrors))
    
conn.commit()

INSERT INTO FoodFacts VALUES ( ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
rowErrors = 48


In [33]:
#PEANUT BUTTER TASK Open sqlite file and retrieve peanut butter information - List the Top 5 Peanut Butters based in 
#Australia and sorted by highest Energy content per 100g

#Based in australia is a little ambiguous - I have interpreted this to mean countries column contains Australia.

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
peanut_frame = pd.read_sql_query('''
SELECT product_name, energy_100g, origins, manufacturing_places, purchase_places, countries 
FROM FoodFacts
''', conn)

In [34]:
peanut_frame = peanut_frame[peanut_frame['product_name'].str.contains("peanut butter", case=False, na=False)]
peanut_frame = peanut_frame[peanut_frame['countries'].str.contains("Australia", case=False, na=False)]
peanut_frame = peanut_frame.loc[:, 'product_name':'energy_100g']
peanut_frame['energy_100g'] = peanut_frame['energy_100g'].replace('', np.nan)
peanut_frame['energy_100g'] = peanut_frame['energy_100g'].astype(float)
peanut_frame = peanut_frame.sort_values(by='energy_100g', ascending=False)
peanut_frame = peanut_frame.head(5)
peanut_frame.to_html('aust_peanuts_with_highest_energy_100g.html')
peanut_frame.head(5)

,product_name,energy_100g
332525,Peanut Butter Crunchy - No added Salt,2670.0
333127,Ridiculously Delicious Peanut Butter Super Smooth,2600.0
332049,natural peanut butter crunchy,2560.0
187095,Bramwells Peanut Butter Crunchy,2530.0
184505,Bramwells American Style Peanut Butter,2470.0


In [36]:
#ZINC TASK Open sqlite file and retrieve zinc information - list the Top 10 Countries together with the number of 
#products with Zinc content above ‘0.001’ and that have more than one product

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
zinc_frame = pd.read_sql_query('''SELECT countries, COUNT(product_name)
FROM FoodFacts 
WHERE zinc_100g IS NOT ""
AND zinc_100g IS NOT NULL 
AND zinc_100g > 0.001
GROUP BY countries
HAVING COUNT(product_name) > 1
ORDER BY COUNT(product_name) DESC
LIMIT 15''', conn)

In [61]:
zf = zinc_frame.head(20)
zf.columns = ['Country', '# of products']
zf.at[zf.index[0], 'Country'] = 'United States'
zf.at[zf.index[7], 'Country'] = 'United States'
zf.at[zf.index[2], 'Country'] = 'Switzerland'
zf.at[zf.index[4], 'Country'] = 'Germany'
zf.at[zf.index[6], 'Country'] = 'Spain'
zf.at[zf.index[8], 'Country'] = 'Russia'
zinc_groups = zf.groupby(zf['Country'])
zf = pd.DataFrame(zinc_groups.sum())
zf = zf.sort_values(by='# of products', ascending=False).head(5)
zf.to_html('Top10_Countries_for_products_with_Zinc.html')
zf.head(5)

,# of products
Country,
United States,2776
France,149
Switzerland,54
Germany,42
Canada,10


In [62]:
# Grouping product categories by those that contain Chicken, Pork and Tofu list their Average, 
# Median and Standard Deviation Protein content per 100g, excluding data that is not available
#(NaN)

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
protein_frame = pd.read_sql_query('''SELECT proteins_100g, ingredients_text
FROM FoodFacts
WHERE proteins_100g IS NOT ""
AND proteins_100g IS NOT "NaN" 
AND proteins_100g IS NOT NULL 
''', conn)

In [76]:
meatDict = {}

chicken_frame = protein_frame[protein_frame['ingredients_text'].str.contains('chicken', case=False, na=False)]
pork_frame = protein_frame[protein_frame['ingredients_text'].str.contains('pork', case=False, na=False)]
tofu_frame = protein_frame[protein_frame['ingredients_text'].str.contains('tofu', case=False, na=False)]

meatDict['chicken'] = chicken_frame
meatDict['pork'] = pork_frame
meatDict['tofu'] = tofu_frame

In [75]:
for k, f in meatDict.items():
    f = pd.DataFrame(f.describe())
    f = f.drop(['count', 'min', '25%', '75%', 'max'])
    last = f.index[-1]
    f = f.rename(index={last: 'median'})
    f.to_html("Average protein content per 100g for {} products.html".format(k))